# CNN Training: Real vs AI-Generated Image Detection

EfficientNetB0 transfer learning and hyperparameter tuning for binary image classification.

This notebook is designed to run on a GPU environment. It loads data from Hugging Face
and saves the trained model back to Hugging Face.

In [ ]:
import datetime
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import keras_tuner as kt
import warnings

from huggingface_hub import ModelCard, ModelCardData, HfApi
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import (
    GlobalAveragePooling2D, GlobalMaxPooling2D,
    Dropout, Dense, BatchNormalization,
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from config import EXPERIMENTS_DIR, MODELS_DIR, LOG_DIR, HF_MODEL_REPO
from utils import load_arrays

warnings.filterwarnings("ignore")

In [ ]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Load Data from Hugging Face

In [ ]:
X_train, y_train = load_arrays("train")
X_val, y_val = load_arrays("validation")
X_test, y_test = load_arrays("test")

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

## TensorBoard Setup

In [ ]:
run_id = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tb_log_dir = str(LOG_DIR / "cnn" / run_id)

tensorboard_cb = TensorBoard(
    log_dir=tb_log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=False,
    update_freq="epoch",
    profile_batch=0,
)

print(f"TensorBoard logs: {tb_log_dir}")

## Stage 1: Train Top Classifier

Freeze the EfficientNetB0 base and train only the classification head.

In [ ]:
# Load base model with ImageNet weights
base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

# Stage 1: Train top only
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)

output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.inputs, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Stage 1: Training top classifier only...")
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=32
)

## Stage 2: Fine-Tune Deeper Layers

Unfreeze the last 100 layers of EfficientNetB0 and fine-tune with a lower learning rate.

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-100]:
    layer.trainable = False

for layer in base_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = False

early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, verbose=1, restore_best_weights=True
)

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Stage 2: Fine-tuning deeper layers...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, tensorboard_cb],
)

In [ ]:
pd.DataFrame(history.history)[['accuracy', 'loss', 'val_accuracy', 'val_loss']].plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.title("Stage 2 Fine-Tuning")
plt.show()

In [ ]:
y_pred_proba = model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype("int32")
print("Validation Results:")
print(classification_report(y_val, y_pred, target_names=["AI", "Human"]))

In [ ]:
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype("int32")
print("Test Results:")
print(classification_report(y_test, y_pred, target_names=["AI", "Human"]))

## Hyperparameter Tuning with Keras Tuner

In [ ]:
class EfficientNetHyperModel(kt.HyperModel):
  def __init__(self, input_shape, num_classes=1):
    self.input_shape = input_shape
    self.num_classes = num_classes

  def build(self, hp):
    base_model = EfficientNetB0(weights='imagenet', include_top=False)
    base_model.trainable = False

    input = tf.keras.Input(shape=self.input_shape, name="our_input_layer")
    augmentation_layer = tf.keras.Sequential([
        tf.keras.layers.RandomFlip(
            mode=hp.Choice('flip_mode', ['horizontal', 'vertical', 'horizontal_and_vertical'])
            ),
        tf.keras.layers.RandomRotation(
            factor=hp.Float("rotation_factor", min_value=0, max_value=.3, step=.1),
            fill_mode=hp.Choice("fill_mode", ['constant', 'reflect', 'wrap', 'nearest']),
            interpolation=hp.Choice("interpolation", ['nearest', 'bilinear'])
            ),
        tf.keras.layers.RandomContrast(
            factor=hp.Float("contrast_factor", min_value=0, max_value=1, step=.2)
        )
    ], name='augmentation_layer')

    x = augmentation_layer(input)
    x = base_model(x, training=False)

    pooling = hp.Choice("pooling", ["avg", "max"])
    if pooling == "avg":
      x = tf.keras.layers.GlobalAveragePooling2D()(x)
    else:
      x = tf.keras.layers.GlobalMaxPooling2D()(x)


    if hp.Boolean("add_dense"):
      x = tf.keras.layers.Dense(
          units=hp.Int("dense_units", min_value=64, max_value=256, step=64),
          activation='relu', name = "our_dense_layer"
      )(x)

    x = tf.keras.layers.Dropout(rate=hp.Float('head_dropout', min_value=0, max_value=.5, step=.1), name="Dropout")(x)

    output = tf.keras.layers.Dense(units=self.num_classes, activation='sigmoid', name="output_layer")(x)

    model = tf.keras.Model(inputs=input, outputs=output)

    lr = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='log')
    optimizer_choice = hp.Choice("optimizer", ["adam", "rmsprop", "adamw"])

    if optimizer_choice == "adam":
      from tensorflow.keras.optimizers import Adam
      optimizer = Adam(learning_rate=lr)
    elif optimizer_choice == "rmsprop":
      from tensorflow.keras.optimizers import RMSprop
      optimizer = RMSprop(learning_rate=lr)
    else:
      from tensorflow.keras.optimizers import AdamW
      optimizer = AdamW(learning_rate=lr, weight_decay=hp.Float('adamw_weight_decay', min_value=1e-7, max_value=.01, sampling='log'))

    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5, name='accuracy'),
                           tf.keras.metrics.AUC(name='AUC')]
                  )

    return model

### Set Up Tuner

In [ ]:
tuner = kt.Hyperband(
    hypermodel=EfficientNetHyperModel(input_shape=(224, 224, 3)),
    objective='val_accuracy',
    seed=10,
    max_epochs=20,
    hyperband_iterations=2,
    executions_per_trial=2,
    directory=str(EXPERIMENTS_DIR),
    project_name='full_hyperparm_tuning'
)

### Search Hyperparameter Space

In [ ]:
tuner_tb = TensorBoard(
    log_dir=str(LOG_DIR / "tuner"),
    histogram_freq=0,
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

tuner.search(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    callbacks=[early_stopping, tuner_tb]
)

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
print("Best hyperparameters:")
print(best_hp.values)

### Rebuild Best Model

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
my_model = EfficientNetHyperModel(input_shape=(224, 224, 3))
best_model = my_model.build(best_hp)
best_model.summary()

In [ ]:
# Optionally load a previously saved model instead
# best_model = tf.keras.models.load_model(MODELS_DIR / "efficient_net_top_model.keras")
# best_model.summary()

### Fine-Tune Best Model

Unfreeze the last 50 layers and fine-tune with early stopping.

In [ ]:
base_model = best_model.get_layer('efficientnetb0')

base_model.trainable = True

for layer in base_model.layers[:-50]:
  layer.trainable = False

In [ ]:
finetune_tb = TensorBoard(
    log_dir=str(LOG_DIR / "cnn" / f"finetune-{run_id}"),
    histogram_freq=1,
    write_graph=True,
    write_images=False,
    update_freq="epoch",
    profile_batch=0,
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

best_model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = best_model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    callbacks=[early_stopping, finetune_tb]
)

In [ ]:
pd.DataFrame(history.history)[['accuracy', 'loss', 'val_accuracy', 'val_loss']].plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.title("Best Model Fine-Tuning")
plt.show()

In [ ]:
y_pred_proba = best_model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype("int32")
print("Test Results:")
print(classification_report(y_test, y_pred, target_names=["AI", "Human"]))

test_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Test AUC: {test_auc:.4f}")

## Save Model to Hugging Face

In [ ]:
# Save model using Keras native HF integration
best_model.save(f"hf://{HF_MODEL_REPO}")
print(f"Model saved to hf://{HF_MODEL_REPO}")

In [ ]:
# Push a detailed model card
card_data = ModelCardData(
    library_name="keras",
    license="mit",
    tags=["image-classification", "ai-generated-image-detection", "efficientnet", "transfer-learning"],
    datasets=["tkbarb10/ADS504-Image-Arrays"],
    model_name="ADS504 EfficientNetB0 - AI vs Human Image Classifier",
)

card_content = f"""---
{card_data.to_yaml()}
---

# ADS504 EfficientNetB0: Real vs AI-Generated Image Classifier

Fine-tuned EfficientNetB0 for binary classification of AI-generated vs human-created images.

## Model Description

- **Architecture:** EfficientNetB0 (transfer learning from ImageNet)
- **Task:** Binary image classification (AI-generated vs Human-created)
- **Input:** 224x224x3 RGB images (uint8)
- **Output:** Sigmoid probability (>0.5 = human)

## Training Procedure

### Stage 1: Train Top Classifier
- Frozen EfficientNetB0 base
- Added: GlobalAveragePooling2D -> Dropout(0.3) -> Dense(1, sigmoid)
- Optimizer: Adam(lr=1e-3)
- Epochs: 5

### Stage 2: Fine-Tune
- Unfroze last 50 layers of EfficientNetB0
- Kept BatchNormalization layers frozen
- Optimizer: Adam(lr=1e-5)
- EarlyStopping: patience=5, restore_best_weights=True
- Max epochs: 50

### Hyperparameter Search (Keras Tuner Hyperband)
Parameters tuned: flip mode, rotation factor, contrast factor, pooling type,
dense layer presence/units, dropout rate, optimizer (Adam/RMSprop/AdamW),
learning rate (1e-5 to 1e-2 log scale).

## Dataset

[tkbarb10/ADS504-Image-Arrays](https://huggingface.co/datasets/tkbarb10/ADS504-Image-Arrays)
- Train: 11,998 images | Validation: 1,499 images | Test: 1,500 images
- Sources: LAION, Open Images, AI vs Human study

## Best Hyperparameters

```json
{json.dumps(best_hp.values, indent=2)}
```

## Metrics

| Metric | Value |
|--------|-------|
| Test AUC | {test_auc:.4f} |

## Limitations

The CNN underperformed classical models (Random Forest: 92% accuracy, AUC 0.98) trained on
hand-crafted image features (color statistics, edge density, texture descriptors).

## Authors

Taylor Kirk, Tommy Baron, Paola Rodriguez - University of San Diego, ADS 504
"""

card = ModelCard(card_content)
card.push_to_hub(HF_MODEL_REPO)
print("Model card pushed.")

In [ ]:
# Upload best hyperparameters as a JSON artifact
hp_path = MODELS_DIR / "best_hyperparameters.json"
hp_path.parent.mkdir(parents=True, exist_ok=True)
with open(hp_path, "w") as f:
    json.dump(best_hp.values, f, indent=2)

api = HfApi()
api.upload_file(
    path_or_fileobj=str(hp_path),
    path_in_repo="best_hyperparameters.json",
    repo_id=HF_MODEL_REPO,
)
print("Hyperparameters uploaded.")

## Launch TensorBoard

From the terminal:
```bash
tensorboard --logdir=logs/
```

Or inline in Jupyter/Colab:
```python
%load_ext tensorboard
%tensorboard --logdir logs/
```